In [1]:
import json
import torch
import torch.nn as nn
from torchtext.legacy.data import Field, Example, Dataset, BucketIterator

#from torchtext.data import Field, Example, Dataset, BucketIterator

# Load and preprocess the dataset
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def preprocess_dataset(dataset):
    examples = []
    for item in dataset:
        question = item['question']
        answer = item['answer']
        example = Example.fromlist([question, answer], fields)
        examples.append(example)
    return examples

# Define your pre-processing and tokenization steps
def tokenize(text):
    return text.split()

# Create the Fields for question and answer
question_field = Field(tokenize=tokenize, lower=True, include_lengths=True)
answer_field = Field(sequential=False, use_vocab=False)

# Define the fields for the dataset
fields = [('question', question_field), ('answer', answer_field)]

# Load and preprocess the training dataset
train_data = load_dataset("sqlData-train.json")
train_examples = preprocess_dataset(train_data)
train_dataset = Dataset(train_examples, fields)

# Load and preprocess the testing dataset
test_data = load_dataset("sqlData-test.json")
test_examples = preprocess_dataset(test_data)
test_dataset = Dataset(test_examples, fields)

# Build the vocabulary
question_field.build_vocab(train_dataset)

# Create the Iterator for batching the data
batch_size = 32  # Adjust the batch size as needed
train_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, test_dataset),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.question),
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# Example of using the word embeddings in a model
class TextToSQLModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(TextToSQLModel, self).__init__()
        self.embedding = nn.Embedding(len(question_field.vocab), embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, answer_field.vocab_size)

    def forward(self, question, question_lengths):
        embedded = self.embedding(question)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, question_lengths.cpu(), enforce_sorted=False)
        _, (hidden, _) = self.lstm(packed)
        output = self.fc(hidden[-1])
        return output

# Initialize the model
embedding_dim = 100  # Adjust the embedding dimension as needed
hidden_dim = 128  # Adjust the hidden dimension as needed
model = TextToSQLModel(embedding_dim, hidden_dim)

# Training loop example
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


model.train()
for epoch in range(num_epochs):
    for batch in train_iterator:
        question, question_lengths = batch.question
        answer = batch.answer

        optimizer.zero_grad()
        output = model(question, question_lengths)
        loss = criterion(output.squeeze(), answer)
        loss.backward()
        optimizer.step()


AttributeError: 'Field' object has no attribute 'vocab_size'